In [ ]:
!pip install datasets evaluate sentencepiece accelerate sentence-transformers

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler, TrainingArguments, Trainer
# from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers import InputExample

from torch.utils.data import DataLoader
from torch.optim import AdamW, SGD
from tqdm.auto import tqdm
import torch
import evaluate
import pandas as pd
import numpy as np
import math
import random
import json
from sklearn.model_selection import train_test_split

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
dataset = {}
df = pd.read_json("/content/drive/MyDrive/shroom/val.model-aware.v2.json")
for ind in df.index:
  df["label"][ind] = 0 if df["label"][ind]=="Hallucination" else 1

dataset["aware"] = df

df = pd.read_json("/content/drive/MyDrive/shroom/val.model-agnostic.json")
for ind in df.index:
  df["label"][ind] = 0 if df["label"][ind]=="Hallucination" else 1

dataset["agnostic"] = df

df = pd.read_json("/content/drive/MyDrive/shroom/test.model-aware.json")

dataset["aware_test"] = df

df = pd.read_json("/content/drive/MyDrive/shroom/test.model-agnostic.json")

dataset["agnostic_test"] = df
# pandas dataframe into huggingface dataset
aware_data = Dataset.from_pandas(dataset["aware"])
agnostic_data = Dataset.from_pandas(dataset["agnostic"])
aware_test_data = Dataset.from_pandas(dataset["aware_test"])
agnostic_test_data = Dataset.from_pandas(dataset["agnostic_test"])

print(aware_data)
print(agnostic_data)

# combine two Datasets into one Dataset not a DatasetDict
all_data = pd.concat([dataset["aware"], dataset["agnostic"]])

print(all_data)


#all_data = all_data.train_test_split(test_size=0.2, seed=42)

#all_data

# put the train and eval data into one dataset
# dataset = DatasetDict({"train": train_data, "eval": eval_data})
# dataset

In [ ]:
def dump_instances(instances, json_file: str):
    json_lst = [ins for ins in instances]
    with open(json_file, "w") as f:
        json.dump(json_lst, f, indent=4)

In [ ]:
def predict(data):
  model = AutoModelForSequenceClassification.from_pretrained(
    'vectara/hallucination_evaluation_model', trust_remote_code=True)
  scores = model.predict([[data[i]["tgt"], data[i]["hyp"]] for i in range(len(data))])

  correct = 0
  for ins in range(len(data)):
    if data[ins]["label"] == 1 and scores[ins] > 0.5:
      correct += 1
    elif data[ins]["label"] == 0 and scores[ins] < 0.5:
      correct += 1

  return correct/len(data)

print("Aware", predict(aware_data))
print("Agnostic", predict(agnostic_data))
print("All", predict(all_data))

In [ ]:
def predict_and_output(data):
  model = AutoModelForSequenceClassification.from_pretrained(
    'vectara/hallucination_evaluation_model', trust_remote_code=True)
  scores = model.predict([[data[i]["tgt"], data[i]["hyp"]] if data[i]["tgt"] != "" else [data[i]["src"], data[i]["hyp"]] for i in range(len(data))])

  docs = []
  for d, score in zip(data, scores):
    pred = dict()
    pred["id"] = d["id"] if "id" in d.keys() else 0
    pred["p(Hallucination)"] = 1 - score
    pred["label"] = "Not Hallucination" if score > 0.5 else "Hallucination"
    docs.append(pred)

  dump_instances(docs, "/content/drive/MyDrive/shroom/output_4.json")
  return docs


a = predict_and_output(aware_test_data)

In [ ]:
def predict_and_dump_model(model, data, output_file=None):
  scores = model.predict([[data[i]["tgt"], data[i]["hyp"]] if data[i]["tgt"] != "" else [data[i]["src"], data[i]["hyp"]] for i in range(len(data))])

  docs = []
  for d, score in zip(data, scores):
    pred = dict()
    pred["id"] = d["id"] if "id" in d.keys() else 0
    pred["p(Hallucination)"] = 1 - score
    pred["label"] = "Not Hallucination" if score > 0.5 else "Hallucination"
    docs.append(pred)
  if output_file is None:
    output_file = "/content/drive/MyDrive/shroom/output_X.json"
  dump_instances(docs, output_file)
  return docs

In [ ]:
def train_and_dump(train_data, num_epochs=5, test_data=None, output_file=None, model=None, intermediate=None):
  if model is None:
    model = AutoModelForSequenceClassification.from_pretrained(
    'vectara/hallucination_evaluation_model', trust_remote_code=True)
    # model = CrossEncoder('vectara/hallucination_evaluation_model', num_labels=1, automodel_args={'ignore_mismatched_sizes': True})
  # manually split data into t_data and test_data after shuffling
  if test_data is None and intermediate is None:
    t_data, test_data = train_test_split(train_data, test_size=0.2, random_state=42)
  else:
    t_data = train_data
  # create input examples for the model
  # if e, use "src" instead of "tgt" if "tgt" is empty
  train_examples = [InputExample(texts=[t_data["tgt"][i], t_data["hyp"][i]], label=int(t_data["label"][i])) if t_data["tgt"][i] != "" else InputExample(texts=[t_data["src"][i], t_data["hyp"][i]], label=int(t_data["label"][i])) for i in range(len(t_data))]
  # test_examples = [InputExample(texts=[test_data["tgt"][i], test_data["hyp"][i]], label=int(test_data["label"][i])) if test_data["tgt"][i] != "" else InputExample(texts=[test_data["src"][i], test_data["hyp"]["i"]], label=int(test_data["label"][i])) for i in range(len(test_data))]
  # test_evaluator = CEBinaryClassificationEvaluator.from_input_examples(test_examples, name='test')

  train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)
  # eval_dataloader = DataLoader(test_examples, batch_size=BATCH_SIZE)
  warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
  model.fit(train_dataloader=train_dataloader,
            epochs=num_epochs,
            warmup_steps=warmup_steps,
            output_path=f"/content/drive/MyDrive/shroom/model",
            use_amp=True,
            show_progress_bar=True)
  if intermediate is None:
    predict_and_dump_model(model, test_data, output_file=output_file)
  return model

train_and_dump(train_data=agnostic_data, num_epochs=3, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_7.json")
train_and_dump(train_data=agnostic_data, num_epochs=4, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_8.json")
train_and_dump(train_data=agnostic_data, num_epochs=5, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_9.json")
train_and_dump(train_data=aware_data, num_epochs=1, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_10.json")
train_and_dump(train_data=aware_data, num_epochs=2, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_11.json")
train_and_dump(train_data=aware_data, num_epochs=3, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_12.json")
train_and_dump(train_data=aware_data, num_epochs=4, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_13.json")
train_and_dump(train_data=aware_data, num_epochs=5, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_14.json")

# joint training

train_and_dump(train_data=all_data, num_epochs=1, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_15.json")
train_and_dump(train_data=all_data, num_epochs=2, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_16.json")
train_and_dump(train_data=all_data, num_epochs=3, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_17.json")
train_and_dump(train_data=all_data, num_epochs=4, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_18.json")
train_and_dump(train_data=all_data, num_epochs=5, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_19.json")

train_and_dump(train_data=all_data, num_epochs=1, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_20.json")
train_and_dump(train_data=all_data, num_epochs=2, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_21.json")
train_and_dump(train_data=all_data, num_epochs=3, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_22.json")
train_and_dump(train_data=all_data, num_epochs=4, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_23.json")
train_and_dump(train_data=all_data, num_epochs=5, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_24.json")

a = train_and_dump(train_data=all_data, num_epochs=3, test_data=None, output_file="/content/drive/MyDrive/shroom/ignore.json", intermediate=True)
train_and_dump(train_data=agnostic_data, num_epochs=1, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_25.json", model=a)

b = train_and_dump(train_data=all_data, num_epochs=3, test_data=None, output_file="/content/drive/MyDrive/shroom/ignore.json", intermediate=True)
train_and_dump(train_data=aware_data, num_epochs=1, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_26.json", model=b)

c = train_and_dump(train_data=all_data, num_epochs=5, test_data=None, output_file="/content/drive/MyDrive/shroom/ignore.json", intermediate=True)
train_and_dump(train_data=agnostic_data, num_epochs=1, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_27.json", model=c)

d = train_and_dump(train_data=all_data, num_epochs=5, test_data=None, output_file="/content/drive/MyDrive/shroom/ignore.json", intermediate=True)
train_and_dump(train_data=aware_data, num_epochs=1, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_28.json", model=d)

e = train_and_dump(train_data=all_data, num_epochs=1, test_data=None, output_file="/content/drive/MyDrive/shroom/ignore.json", intermediate=True)
train_and_dump(train_data=agnostic_data, num_epochs=1, test_data=agnostic_test_data, output_file="/content/drive/MyDrive/shroom/output_29.json", model=e)

f = train_and_dump(train_data=all_data, num_epochs=1, test_data=None, output_file="/content/drive/MyDrive/shroom/ignore.json", intermediate=True)
train_and_dump(train_data=aware_data, num_epochs=1, test_data=aware_test_data, output_file="/content/drive/MyDrive/shroom/output_30.json", model=f)



In [ ]:
def predict_model(model, data):
  scores = model.predict([[data["tgt"][i], data["hyp"][i]] for i in data.index])

  correct = 0
  for ins, s_ins in zip(data.index, range(len(scores))):
    if data["label"][ins] == 1 and scores[s_ins] > 0.5:
      correct += 1
    elif data["label"][ins] == 0 and scores[s_ins] < 0.5:
      correct += 1

  print(correct/len(data))